# Packages

In [1]:
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import torch
import segmentation_models_pytorch as smp
from PIL import Image
import albumentations as albu
from torch import nn
from PIL import Image

# Data

In [2]:
list_image = glob.glob("C:\\Users\\gueganj\\Desktop\\My_DataBase\\nature\\images\\*.jpg")

In [3]:
input_size = (544,960)

# Inpainting

In [4]:
def my_inpainting(img, mask, W=10, reduce='median'):
    new_img = img.copy()
    Nr, Nc = img.shape[0], img.shape[1]
    r, c = np.where(mask)
    for i in range(len(r)):
        bottom_left  = max(0,r[i]-W)
        bottom_right = min(Nr,r[i]+W)
        up_left      = max(0,c[i]-W)
        up_right     = min(Nc,c[i]+W)
        window_msk = mask[bottom_left:bottom_right,up_left:up_right]
        window_img = img[bottom_left:bottom_right,up_left:up_right]
        while len(window_img[window_msk==0]) == 0: # if there is only mask in window
            bottom_left  = max(0,bottom_left-1)
            bottom_right = min(Nr,bottom_right+1)
            up_left      = max(0,up_left-1)
            up_right     = min(Nc,up_right+1)
            window_msk = mask[bottom_left:bottom_right,up_left:up_right]
            window_img = img[bottom_left:bottom_right,up_left:up_right]
        if reduce == 'median': # less sensitive to window size
            new_pixel  = np.median(window_img[window_msk==0], axis=0)
        elif reduce == 'mean':
            new_pixel  = np.mean(window_img[window_msk==0], axis=0)
        new_img[r[i],c[i]] = new_pixel
    return new_img

In [5]:
list_image = glob.glob('C:\\Temp\\data\\video_image_4\\*.jpg')

In [10]:
root = 'C:\\Users\\gueganj\\Desktop\\My_DataBase\\nature\\'
image_path = os.path.join(root,'images','021_scanRecordPD_0.jpg')
image_name = os.path.basename(image_path)
img  = cv2.imread(image_path)
img  = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#img  = cv2.resize(img, (input_size[1],input_size[0]))
mask = np.array(Image.open(root + 'masks\\frame\\' + image_name.replace('.jpg','.png')).convert('P'))
# our labellisation is too small, dilate mask
kernel = np.ones((4,4), np.uint8)
mask   = cv2.dilate(mask, kernel, iterations=1)
img_inpaint = cv2.inpaint(img, mask, 10, cv2.INPAINT_TELEA) # INPAINT_NS INPAINT_TELEA
plt.imsave('C:\\Users\\gueganj\\Desktop\\'+image_name,img_inpaint)

In [29]:
image_id = ''
for i, image_path in enumerate(tqdm(list_image)):
    image_name = os.path.basename(image_path)
    img  = cv2.imread(image_path)
    img  = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img  = cv2.resize(img, (input_size[1],input_size[0]))
    mask = np.array(Image.open("C:\\Temp\\data\\mask_image_4\\" + image_name.replace('.jpg','.png')).convert('P'))
    # our labellisation is too small, dilate mask
    kernel = np.ones((4,4), np.uint8)
    mask   = cv2.dilate(mask, kernel, iterations=1)
    img_inpaint = cv2.inpaint(img, mask, 10, cv2.INPAINT_TELEA) # INPAINT_NS INPAINT_TELEA
    plt.imsave('C:\\Temp\\data\\inpainting_4\\'+image_name,img_inpaint)

100%|████████████████████████████████████████████████████████████████████████████████| 376/376 [01:59<00:00,  3.14it/s]


# On a video

In [5]:
# prepare data
#outdir = "C:\\Temp\\data\\inpainting_1"

In [6]:
def resize(input_size, deform="rectangular"):
    """Transformation for validation set
    input : 
        - input_size : integer for resizing maximum side
    output :
        - transform : albumentations object
    """
    if deform=="square":
        transform = albu.Resize(input_size, input_size)
    elif deform=="scale":
        transform = albu.LongestMaxSize(input_size)
    elif deform=="rectangular":
        transform = albu.Resize(input_size[0], input_size[1])
    else:
        print("deform argument unknown")
        
    return transform

# Do inpainting - change color - change frame

In [380]:
video_name = 'C:\\Temp\\data\\change_frame_4.avi'
image_path = 'C:\\Temp\\data\\video_image_4\\'
inpainting_path = 'C:\\Temp\\data\\inpainting_4\\'
mask_path = 'C:\\Temp\\data\\mask_image_4\\'
eye_rgba   = np.array(Image.open('eyeglasses.png'))
eye_shape  = eye_rgba.shape

In [381]:
fourcc     = cv2.VideoWriter_fourcc(*'DIVX')
video      = cv2.VideoWriter(video_name, fourcc, 15, (input_size[1],input_size[0]))
list_image = glob.glob(os.path.join(image_path,"*.jpg"))
cpt = 0
for i in tqdm(range(len(list_image))):
    if i % 50 == 0:
        cpt = cpt + 1
        if (cpt % 4 == 1): print("original")
        if (cpt % 4 == 2): print("change color")
        if (cpt % 4 == 3): print("inpainting")
        if (cpt % 4 == 0): print("change frame")
    cpt = 4
    # get mask if needed        
    if (cpt % 4 == 2) or (cpt % 4 == 0):
        mask_name = os.path.join(mask_path, str(i)+'.png')
        mask = np.array(Image.open(mask_name).convert('P'))
    
    # read image
    img = plt.imread(os.path.join(image_path, str(i)+'.jpg'))
    # resize
    img = cv2.resize(img, (input_size[1],input_size[0]))
   
    # inpainting
    if (cpt % 4 == 3) or (cpt % 4 == 0):
        img = plt.imread(os.path.join(inpainting_path,str(i)+'.jpg'))

    # new color
    if (cpt % 4 == 2):
        img[mask==1,:] = img[mask==1,:] + [80,-20,-20]
        
    # new frame
    if (cpt % 4 == 0):
        # perspective
        mask[400:,:] = 0 #filter
        br,bl,ur,ul = get_box_eyeglasses(mask)
        dst = np.float32([ul,ur,bl,br])
        src = np.float32([[0, 0], [eye_rgba.shape[1] - 1, 0], [0, eye_rgba.shape[0] - 1], [eye_rgba.shape[1] - 1, eye_rgba.shape[0] - 1]])
        # conserve +- heigth/width proportion
        h1, w1 = src[2,1]-src[0,1], src[1,0]-src[0,0]
        h2, w2 = dst[2,1]-dst[0,1], dst[1,0]-dst[0,0]
        h      = (h1/w1)*w2
        dst[2,1] = dst[0,1] + round(5*h/4) # work here because new eyeglasses are smaller
        dst[3,1] = dst[1,1] + round(5*h/4) # otherwise substraction might be needed
        dst[0,1] = dst[0,1] + round(h/4)
        dst[1,1] = dst[1,1] + round(h/4)
        M = cv2.getPerspectiveTransform(src,dst)
        eye_perspective = cv2.warpPerspective(eye_rgba, M, (mask.shape[1],mask.shape[0]))
        # merge with alpha channel
        alpha_eyeglasses = np.expand_dims(eye_perspective[:,:,3]/255, axis=2)
        alpha_original   = 1.0 - alpha_eyeglasses
        img_merge = (alpha_eyeglasses * eye_perspective[:,:,:3]).astype(int) + (alpha_original * img).astype(int)
        # get back to full image
        img = img.copy()
        img =  np.uint8(img_merge)

    video.write(cv2.cvtColor(img,cv2.COLOR_RGB2BGR))
video.release()

  0%|▏                                                                                 | 1/376 [00:00<00:44,  8.42it/s]

original


 14%|██████████▉                                                                      | 51/376 [00:05<00:36,  8.86it/s]

original


 27%|█████████████████████▍                                                          | 101/376 [00:11<00:29,  9.25it/s]

...

 80%|████████████████████████████████████████████████████████████████                | 301/376 [00:33<00:09,  7.78it/s]

original


 93%|██████████████████████████████████████████████████████████████████████████▋     | 351/376 [00:39<00:02,  9.37it/s]

original


100%|████████████████████████████████████████████████████████████████████████████████| 376/376 [00:43<00:00,  8.70it/s]


# Perspective eyeglasses

In [328]:
def get_box_eyeglasses(mask):
    # rectangle box
    idx          = np.where(mask)
    x_max, x_min = idx[0].max(), idx[0].min()
    y_max, y_min = idx[1].max(), idx[1].min()
    mask_w = mask[x_min:x_max,y_min:y_max]
    # small parallelogram box
    y_max_1 = int(np.min(np.where(mask[x_max,:]))) # use min ou max ou median ?
    y_min_1 = int(np.max(np.where(mask[x_min,:])))
    x_max_1, x_max_2 = max(np.where(mask[:,y_max_1])[0]), min(np.where(mask[:,y_max_1])[0])
    x_min_1, x_min_2 = max(np.where(mask[:,y_min_1])[0]), min(np.where(mask[:,y_min_1])[0])
    # final box
    coeff  = (x_min_1-x_max_1)/(y_min_1-y_max_1)
    offset = x_min_1-coeff*y_min_1
    x1, x2 = round(coeff*y_max+offset), round(coeff*y_min+offset)
    coeff  = (x_min_2-x_max_2)/(y_min_1-y_max_1)
    offset = x_min_2-coeff*y_min_1
    x3, x4 = round(coeff*y_max+offset), round(coeff*y_min+offset)

    return (y_max,x1),(y_min,x2),(y_max,x3),(y_min,x4)